<a href="https://colab.research.google.com/github/toddwalters/pgaiml-python-coding-examples/blob/main/capstone/projects/historicalStructureClassification/1703138137_ToddWalters_project_historicalStructureClassification_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Table of Contents

-----

# <a id='toc1_'></a>[**Historical Structure Classification Project**](#toc0_)

-----------------------------
## <a id='toc1_1_'></a>[**Project Context**](#toc0_)
-----------------------------

This project is part of an AIML Capstone and focuses on two main aspects:
1. Classifying historical structures using deep learning techniques
2. Developing a recommendation engine for tourism

The project aims to assist the travel and tourism industries by monitoring the condition of historical structures and providing personalized recommendations to tourists.

-----------------------------
## <a id='toc1_2_'></a>[**Project Objectives**](#toc0_)
-----------------------------

1. Develop a CNN model to classify historical structures from images
2. Perform exploratory data analysis on tourism data
3. Create a recommendation engine for tourist attractions

-----------------------------
## <a id='toc1_3_'></a>[**Project Dataset Description**](#toc0_)
-----------------------------

**Part 1: Image Classification**

1. **Structures_dataset.zip**: Training set of images of historical structures
2. **dataset_test**: Test set of images of historical structures

**Part 2: Tourism Recommendation**

1. **user.csv**: User demographic data
   - Columns: User_id, location, age
2. **tourism_with_id.csv**: Details on tourist attractions in Indonesia's five largest cities
   - Columns: Place_id, Place_name, Description, Category, City, Price, Rating, Time_minute, Coordinate, Lat, Long
3. **tourism_rating.csv**: User ratings for tourist attractions
   - Columns: user_id, place_id, place_rating

-----------------------------------
## <a id='toc1_4_'></a>[**Project Analysis Steps To Perform**](#toc0_)
-----------------------------------


**Part 1 - Deep Learning**

1. Plot the sample images (8–10) from each class or category to gain a better understanding
of each class
   > Hint: You can use the OpenCV open-source library for this task.
2. Select an CNN architecture of your choice to train the CV model. Configure the
architecture for transfer learning, set up a TensorFlow environment for the selected
backbone architecture, and load pre-trained weights
   > Note: Algorithm or architecture selection is an important step in the training of ML models,
so select the one that performs the best on your dataset.
3. Deep learning models tend to work well with large amounts (millions of images) of data, but
we may not always have enough data to train a deep learning model from scratch. Transfer
learning techniques allow us to train and fine-tune large deep learning architectures using
limited data.
   > Hint: For transfer learning, use pre-trained CNN weights and freeze all convolutional
layers' weights.
4. As of now, CNN architecture has been configured for our model. Modify the top of this
architecture to work with our dataset by:
• Adding an appropriate number of dense layers with an activation function.
• Using dropout for regularization.
   > Note: It is important to understand that these parameters are hyperparameters that
must be tuned.
5. Compile the model with the right set of parameters like optimizer, loss function, and metric
6. Define your callback class to stop the training once validation accuracy reaches a certain
number of your choice
7. Setup the train or test dataset directories and review the number of image samples for the train
and test datasets for each class
8. Train the model without augmentation while continuously monitoring the validation accuracy
9. Train the model with augmentation and keep monitoring validation accuracy
   > Note: Choose carefully the number of epochs, steps per epoch, and validation steps based on
your computer configuration
10. Visualize training and validation accuracy on the y-axis against each epoch on the x-axis to
see if the model overfits after a certain epoch
Deep learning

**Part 2 - Data Science**

1. Import all the datasets and perform preliminary inspections, such as:
   1. Check for missing values and duplicates
   2. Remove any anomalies found in the data
2. To understand the tourism highlights better, we should explore the data in depth.
   1. Explore the user group that provides the tourism ratings by:
      - Analyzing the age distribution of users visiting the places and rating them
      - Identifying the places where most of these users (tourists) are coming from
3. Next, let's explore the locations and categories of tourist spots.
   1. What are the different categories of tourist spots?
   2. What kind of tourism each location is most famous or suitable for?
   3. Which city would be best for a nature enthusiast to visit?
4. To better understand tourism, we need to create a combined data with places and their user ratings.
   1. Use this data to figure out the spots that are most loved by the tourists. Also, which city
has the most loved tourist spots?
   2. Indonesia provides a wide range of tourist spots ranging from historical and cultural
beauties to advanced amusement parks. Among these, which category of places are users
liking the most?
5. Build a recommender model for the system
   1. Use the above data to develop a collaborative filtering model for recommendation and
use that to recommend other places to visit using the current tourist location(place name)



### <a id='toc1_4_1_'></a>[**Part 1 - Deep Learning**](#toc0_)

#### <a id='toc1_4_1_1_'></a>[**Import Modules and Set Default Environment Variables**](#toc0_)

In [ ]:
import gc
import math
import multiprocessing
import os
import time
import warnings
from concurrent.futures import ThreadPoolExecutor

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.applications import (EfficientNetB0, InceptionV3, MobileNetV2, ResNet50V2, VGG16)
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.callbacks import (Callback, EarlyStopping, LambdaCallback, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.layers import (BatchNormalization, Dense, Dropout, GlobalAveragePooling2D, Input)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence
warnings.filterwarnings("ignore")

# Enable if having issues with Tensorflow
# os.environ['TF_DISABLE_GRAPPLER'] = '1'

# Load the environment variables
# load_dotenv(verbose=True, dotenv_path='.env', override=True)
# DATASET_PATH = f'{DATASET_PATH}/structures_dataset'

# Uncomment when using Google Colab and Mounting Google Drive
DATASET_PATH = '/content/drive/MyDrive/Colab Notebooks/datasets/1703138137_dataset/part_1/dataset_hist_structures'

# Mount Google Drive if using Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    DATASET_PATH = '/content/drive/MyDrive/Colab Notebooks/datasets/1703138137_dataset/part_1/dataset_hist_structures'
except ImportError:
    DATASET_PATH = '/Users/toddwalters/Library/CloudStorage/GoogleDrive-toddw4271@gmail.com/My Drive/Colab Notebooks/datasets/1703138137_dataset/part_1/dataset_hist_structures'

# Check TensorFlow Verision
print()
print(f"TensorFlow version: {tf.__version__}")
print(f"Eager execution: {tf.executing_eagerly()}")
print()


#### <a id='toc1_4_1_1_'></a>[**Look for corrupt files**](#toc0_)

In [ ]:
def verify_images(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                file_path = os.path.join(root, file)
                try:
                    img = Image.open(file_path) # open the image file
                    img.verify() # verify that it is, in fact an image
                except (IOError, SyntaxError) as e:
                    print(f'Bad file: {file_path}') # print out the names of corrupt files
                    os.remove(file_path)
                    print(f'Deleted bad file: {file_path}')

# verify_images(f'{DATASET_PATH}/dataset_test')
# verify_images(f'{DATASET_PATH}/structure_dataset')

#### <a id='toc1_4_1_1_'></a>[**Rename Files**](#toc0_)

In [ ]:
import os
import uuid
import random
import string

def random_string(length):
    """Generate a random string of fixed length"""
    letters = string.ascii_lowercase + string.digits
    return ''.join(random.choice(letters) for i in range(length))

def rename_files(directory):
    for root, dirs, files in os.walk(directory):
        for filename in files:
            # Generate a unique name
            new_name = str(uuid.uuid4())

            # Get the file extension
            file_extension = os.path.splitext(filename)[1]

            # Create the new filename
            new_filename = f"{new_name}.jpg"

            # Full paths
            old_file = os.path.join(root, filename)
            new_file = os.path.join(root, new_filename)

            # Rename the file
            os.rename(old_file, new_file)
            print(f"Renamed: {filename} -> {new_filename}")

# Uncomment if you want to rename all of the files in the dataset
#
# Specify the directory to start from
# start_directory = f'{DATASET_PATH}/structures_dataset'
# Run the function
# rename_files(start_directory)
# Specify the directory to start from
# start_directory = f'{DATASET_PATH}/dataset_test'
# # Run the function
# rename_files(start_directory)

#### <a id='toc1_4_1_1_'></a>[**Plot The Sample Images**](#toc0_)

In [ ]:
def plot_sample_images(dataset_path, num_samples=8):
    classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    fig, axes = plt.subplots(len(classes), num_samples, figsize=(20, 4*len(classes)))

    for i, class_name in enumerate(classes):
        class_path = os.path.join(dataset_path, class_name)
        images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))][:num_samples]

        for j, image_name in enumerate(images):
            image_path = os.path.join(class_path, image_name)
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            axes[i, j].imshow(img)
            axes[i, j].axis('off')

        # Set the class name as the title for the first image in the row
        axes[i, 0].set_title(class_name, fontsize=16, pad=20)

    plt.tight_layout()
    plt.show()

plot_sample_images(f'{DATASET_PATH}/structures_dataset')

**Explanations:**

- This code defines a function `plot_sample_images` that takes a dataset path and plots a specified number of sample images from each class. It uses OpenCV to read the images and matplotlib to display them in a grid.

**Why It Is Important:**

- Visualizing sample images from each class helps us understand the nature of the data we're working with. It allows us to identify any potential issues with the images, such as inconsistent sizes, color schemes, or quality. This step is crucial for determining if any preprocessing steps are needed before training the model.

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_1_1_'></a>[**Define Utility Functions and Classes**](#toc0_)

In [ ]:
# Define utility functions and classes
class DataGenerator(Sequence):
    def __init__(self, generator, steps_per_epoch):
        self.generator = generator
        self.steps_per_epoch = steps_per_epoch

    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, index):
        return next(self.generator)

    def on_epoch_end(self):
        self.generator.reset()

def count_samples(dataset_path):
    class_counts = {}
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            class_counts[class_name] = len(os.listdir(class_path))
    return class_counts

class DebugCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print()
        print(f"\nStarting epoch {epoch + 1}")
        print()

    def on_batch_begin(self, batch, logs=None):
        if batch % 50 == 0:
            print()
            print(f"Starting batch {batch}")
            print()

    def on_epoch_end(self, epoch, logs=None):
        print()
        print(f"End of epoch {epoch + 1}")
        print()
        if logs:
            for key, value in logs.items():
                print()
                print(f"{key}: {value}")
                print()
        print()
        print("--------------------")
        print()

def create_data_generators(train_path, test_path, batch_size=32):
    train_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
        train_path,
        # target_size=(128, 128),
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    test_generator = test_datagen.flow_from_directory(
        test_path,
        # target_size=(128, 128),
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    steps_per_epoch = len(train_generator)
    validation_steps = len(test_generator)

    return DataGenerator(train_generator, steps_per_epoch), DataGenerator(test_generator, validation_steps), steps_per_epoch, validation_steps

class DatasetLogger(Callback):
    def __init__(self, train_generator, val_generator):
        super().__init__()
        self.train_generator = train_generator
        self.val_generator = val_generator

    def on_epoch_begin(self, epoch, logs=None):
        print()
        print(f"Epoch {epoch + 1} - Train samples: {len(self.train_generator.generator.classes)}")
        print(f"Epoch {epoch + 1} - Val samples: {len(self.val_generator.generator.classes)}")
        print()
        print("--------------------")
        print()

    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            print()
            print(f"Epoch {epoch + 1} - Train accuracy: {logs.get('accuracy', 'N/A'):.4f}")
            print(f"Epoch {epoch + 1} - Val accuracy: {logs.get('val_accuracy', 'N/A'):.4f}")
            print()
            print("--------------------")
            print()

class CustomValidationCallback(Callback):
    def __init__(self, validation_data, validation_steps):
        super().__init__()
        self.validation_data = validation_data
        self.validation_steps = validation_steps

    def on_epoch_end(self, epoch, logs=None):
        val_loss = 0
        val_accuracy = 0
        for i in range(self.validation_steps):
            x, y = next(self.validation_data.generator)
            val_metrics = self.model.test_on_batch(x, y)
            val_loss += val_metrics[0]
            val_accuracy += val_metrics[1]
        val_loss /= self.validation_steps
        val_accuracy /= self.validation_steps
        logs['val_loss'] = val_loss
        logs['val_accuracy'] = val_accuracy
        print()
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        print()

#### <a id='toc1_4_1_2_'></a>[**Select an CNN architecture**](#toc0_)

In [ ]:
def create_model(num_classes):
    # base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    # inputs = Input(shape=(128, 128, 3))
    inputs = Input(shape=(224, 224, 3))
    x = CustomAugmentationLayer()(inputs)
    x = base_model(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = Dropout(0.5)(x)
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=output)
    return model

**Explanations:**

- This code defines a function `create_model` that creates a CNN using transfer learning. It uses ResNet50 as the base model with pre-trained ImageNet weights. The base model layers are frozen, and custom dense layers are added on top for fine-tuning.

**Why It Is Important:**

- Transfer learning allows us to leverage pre-trained models on large datasets, which is particularly useful when we have limited data. By using a pre-trained model as a feature extractor and adding custom layers, we can adapt the model to our specific classification task while benefiting from the general features learned by the base model.

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_1_3_'></a>[**Compile The Model**](#toc0_)

In [ ]:
def compile_model(model):
    initial_learning_rate = 0.001
    lr_schedule = ExponentialDecay(
        initial_learning_rate,
        # decay_steps=1000,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True)

    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

**Explanations:**

- This code defines a function `compile_model` that compiles the model with the Adam optimizer, categorical crossentropy loss function, and accuracy metric.

**Why It Is Important:**

- Compiling the model is a crucial step that defines how the model will be trained. The choice of optimizer, loss function, and metrics affects the training process and the model's performance. Categorical crossentropy is appropriate for multi-class classification tasks, and accuracy provides a clear measure of the model's performance.

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_2_'></a>[**Define Callback Class**](#toc0_)

In [ ]:
class AccuracyCallback(Callback):
    def __init__(self, target_accuracy):
        super().__init__()
        self.target_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_accuracy') >= self.target_accuracy:
            print()
            print(f"\nReached {self.target_accuracy*100}% validation accuracy. Stopping training.")
            print()
            print("--------------------")
            print()
            self.model.stop_training = True

**Explanations:**

- This code defines a custom callback class `AccuracyCallback` that stops the training when a target validation accuracy is reached.

**Why It Is Important:**

- Callbacks allow us to customize the training process. In this case, we're using a callback to prevent overfitting by stopping the training once we've reached a satisfactory level of validation accuracy. This helps us avoid wasting computational resources and reduces the risk of the model memorizing the training data.

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_2_1_'></a>[**Set Up Dataset Directories And Review Sample Numbers**](#toc0_)

In [ ]:
# Setup Dataset Directories and Review Sample Numbers
train_path = f'{DATASET_PATH}/structures_dataset'
test_path = f'{DATASET_PATH}/dataset_test'

train_counts = count_samples(train_path)
test_counts = count_samples(test_path)

print()
print("\nTraining samples per class:")
for class_name, count in sorted(train_counts.items()):
    print(f"{class_name}: {count}")

print("\nTest samples per class:")
for class_name, count in sorted(test_counts.items()):
    print(f"{class_name}: {count}")

print()

# Create and compile the model
num_classes = len(train_counts)
model = create_model(num_classes)

print(model.summary())

compiled_model = compile_model(model)

# Set up data generators
batch_size = 32
train_generator, test_generator, steps_per_epoch, validation_steps = create_data_generators(train_path, test_path, batch_size)

print()
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")
print()

**Explanations:**

- This code defines a function `count_samples` that counts the number of samples in each class for a given dataset. It then applies this function to both the training and test datasets and prints the results.

**Why It Is Important:**

- Understanding the distribution of samples across classes is crucial for several reasons:

    1. It helps identify any class imbalance issues that may need to be addressed.
    2. It ensures we have enough samples in each class for both training and testing.
    3. It helps in setting appropriate batch sizes and steps per epoch during training.

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_2_2_'></a>[**Train The Model Without Augmentation**](#toc0_)

In [ ]:
# Main execution
if __name__ == "__main__":
    # Set up callbacks
    accuracy_callback = AccuracyCallback(target_accuracy=0.95)
    debug_callback = DebugCallback()
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    dataset_logger = DatasetLogger(train_generator, test_generator)
    custom_validation = CustomValidationCallback(test_generator, validation_steps)

    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.generator.classes), y=train_generator.generator.classes)
    class_weight_dict = dict(enumerate(class_weights))

    try:
        # Separate evaluation before training
        model.summary
        print()
        print("Evaluating model before training:")
        evaluation = compiled_model.evaluate(test_generator.generator, steps=validation_steps)
        print(f"Initial evaluation: {evaluation}")
        print()

        # Training
        history = compiled_model.fit(
            train_generator.generator,
            steps_per_epoch=steps_per_epoch,
            epochs=20,
            validation_data=test_generator.generator,
            validation_steps=validation_steps,
            callbacks=[early_stopping, debug_callback, dataset_logger, custom_validation],
            class_weight=class_weight_dict,
            verbose=1
        )

    except tf.errors.ResourceExhaustedError:
        print()
        print("Memory error occurred. Try reducing batch size or image dimensions.")
        print()
    except Exception as e:
        print()
        print(f"An error occurred during training: {e}")
        print()
        import traceback
        traceback.print_exc()

    # Final evaluation
    print()
    print("Final evaluation:")
    print("-----------------")
    final_evaluation = compiled_model.evaluate(test_generator.generator, steps=validation_steps)
    print()
    print(f"Final evaluation: {final_evaluation}")
    print()

    # Plot training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


**Explanations:**

- This code sets up data generators for the training and test datasets, then trains the model using these generators. The `ImageDataGenerator` is used to load and preprocess images in batches, which is memory-efficient for large datasets.

**Why It Is Important:**

- Training the model without augmentation provides a baseline performance. It allows us to see how well the model performs with the original data before applying any data augmentation techniques. This step is crucial for understanding if the model has enough capacity to learn from the data and if there are any immediate issues like overfitting or underfitting.

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_2_3_'></a>[**Train The Model With Augmentation**](#toc0_)

In [ ]:
import gc
import math
import os
import warnings
from dotenv import load_dotenv

# import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.applications import (EfficientNetB0, InceptionV3, MobileNetV2, ResNet50V2, VGG16)
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.resnet_v2 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.callbacks import (Callback, EarlyStopping, LambdaCallback, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.layers import (BatchNormalization, Dense, Dropout, GlobalAveragePooling2D, Input)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence

# To ignore warnings
warnings.filterwarnings("ignore")

class AccuracyCallback(Callback):
    def __init__(self, target_accuracy):
        super().__init__()
        self.target_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs=None):
        if logs.get('val_accuracy') >= self.target_accuracy:
            print(f"\nReached {self.target_accuracy*100}% validation accuracy. Stopping training.")
            print()
            print("--------------------")
            print()
            self.model.stop_training = True

class AugmentationCallback(Callback):
    def __init__(self, train_generator):
        super().__init__()
        self.train_generator = train_generator

class CustomAugmentationLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(CustomAugmentationLayer, self).__init__(**kwargs)

    def call(self, inputs, training=None):
        if training:
            augmented = tf.image.random_contrast(inputs, lower=0.8, upper=1.2)
            augmented = tf.image.random_saturation(augmented, lower=0.8, upper=1.2)
            return augmented
        return inputs

class CustomValidationCallback(Callback):
    def __init__(self, validation_data, validation_steps):
        super().__init__()
        self.validation_data = validation_data
        self.validation_steps = validation_steps

    def on_epoch_end(self, epoch, logs=None):
        val_loss = 0
        val_accuracy = 0
        for i in range(self.validation_steps):
            x, y = next(self.validation_data.generator)
            val_metrics = self.model.test_on_batch(x, y)
            val_loss += val_metrics[0]
            val_accuracy += val_metrics[1]
        val_loss /= self.validation_steps
        val_accuracy /= self.validation_steps
        logs['val_loss'] = val_loss
        logs['val_accuracy'] = val_accuracy
        print()
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        print()

class DataGenerator(Sequence):
    def __init__(self, generator, steps_per_epoch):
        self.generator = generator
        self.steps_per_epoch = steps_per_epoch

    def __len__(self):
        return self.steps_per_epoch

    def __getitem__(self, index):
        return next(self.generator)

    def on_epoch_end(self):
        self.generator.reset()

class DatasetLogger(Callback):
    def __init__(self, train_generator, val_generator):
        super().__init__()
        self.train_generator = train_generator
        self.val_generator = val_generator

    def on_epoch_begin(self, epoch, logs=None):
        print()
        print(f"Epoch {epoch + 1} - Train samples: {len(self.train_generator.generator.classes)}")
        print(f"Epoch {epoch + 1} - Val samples: {len(self.val_generator.generator.classes)}")
        print()
        print("--------------------")
        print()

    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            print()
            print(f"Epoch {epoch + 1} - Train accuracy: {logs.get('accuracy', 'N/A'):.4f}")
            print(f"Epoch {epoch + 1} - Val accuracy: {logs.get('val_accuracy', 'N/A'):.4f}")
            print()
            print("--------------------")
            print()

class DebugCallback(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print()
        print(f"\nStarting epoch {epoch + 1}")
        print()

    def on_batch_begin(self, batch, logs=None):
        if batch % 50 == 0:
            print()
            print(f"Starting batch {batch}")
            print()

    def on_epoch_end(self, epoch, logs=None):
        print()
        print(f"End of epoch {epoch + 1}")
        print()
        if logs:
            for key, value in logs.items():
                print(f"{key}: {value}")
        print()
        print("--------------------")
        print()

class LearningRateFinder:
    def __init__(self, model, stop_factor=4, beta=0.98):
        self.model = model
        self.stop_factor = stop_factor
        self.beta = beta
        self.lrs = []
        self.losses = []
        self.best_loss = 1e9
        self.avg_loss = 0
        self.batch_num = 0
        self.weightsFile = None

    def reset(self):
        self.lrs = []
        self.losses = []
        self.best_loss = 1e9
        self.avg_loss = 0
        self.batch_num = 0
        self.weightsFile = None

    def on_batch_end(self, batch, logs):
        lr = self.model.optimizer.lr
        if hasattr(lr, 'numpy'):
            lr = lr.numpy()
        self.lrs.append(lr)

        loss = logs["loss"]
        self.batch_num += 1
        self.avg_loss = (self.beta * self.avg_loss) + ((1 - self.beta) * loss)
        smooth = self.avg_loss / (1 - (self.beta ** self.batch_num))
        self.losses.append(smooth)

        stop_loss = self.stop_factor * self.best_loss

        if self.batch_num > 1 and smooth < self.best_loss:
            self.best_loss = smooth

        if smooth > stop_loss:
            self.model.stop_training = True

        if self.batch_num % 10 == 0:  # Print every 10 batches
            print(f"Batch {self.batch_num}: lr = {lr:.6f}, loss = {smooth:.4f}")

    def find(self, train_data, start_lr, end_lr, batch_size=32, epochs=5):
        self.reset()
        num_samples = len(train_data.generator.classes)
        steps_per_epoch = math.ceil(num_samples / batch_size)

        self.weightsFile = "lrf_weights.h5"
        self.model.save_weights(self.weightsFile)

        orig_lr = self.model.optimizer.lr.numpy()
        self.model.optimizer.lr.assign(start_lr)

        print()
        print(f"Starting Learning Rate Finder: LR range {start_lr} to {end_lr}")
        print()

        lr_schedule = lambda epoch, batch: start_lr * (end_lr / start_lr) ** (batch / (epochs * steps_per_epoch))

        callback = LambdaCallback(on_batch_end=lambda batch, logs: self.on_batch_end(batch, logs))
        self.model.fit(train_data.generator,
                       steps_per_epoch=steps_per_epoch,
                       epochs=epochs,
                       callbacks=[callback, LearningRateScheduler(lr_schedule)],
                       verbose=0)

        print()
        print("Learning Rate Finder complete")
        print()

        self.model.load_weights(self.weightsFile)
        self.model.optimizer.lr.assign(orig_lr)

    def plot_loss(self):
        plt.figure(figsize=(10, 6))
        plt.plot(self.lrs, self.losses)
        plt.xscale("log")
        plt.xlabel("Learning rate")
        plt.ylabel("Loss")
        plt.title("Learning Rate vs. Loss")
        plt.show()

    def get_best_lr(self):
        min_grad_idx = np.argmin(np.gradient(np.array(self.losses)))
        best_lr = self.lrs[min_grad_idx]
        print()
        print(f"Best initial learning rate found: {best_lr:.6f}")
        print()
        return best_lr

    def print_results(self):
        print()
        print("\nLearning Rate Finder Results:")
        print(f"Minimum loss: {min(self.losses):.4f}")
        print(f"Maximum loss: {max(self.losses):.4f}")
        print(f"Learning rate range: {min(self.lrs):.6f} to {max(self.lrs):.6f}")
        print()

def count_samples(dataset_path):
    class_counts = {}
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            class_counts[class_name] = len(os.listdir(class_path))
    return class_counts

def create_model(num_classes, architecture, input_shape):

    base_model = architecture(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Freeze the base model layers
    for layer in base_model.layers:
        layer.trainable = False

    inputs = Input(shape=input_shape)
    x = CustomAugmentationLayer()(inputs)
    x = base_model(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = Dropout(0.5)(x)
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=output)
    return model

def compile_model(model, learning_rate):
    lr_schedule = ExponentialDecay(
        learning_rate,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True)

    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'],
                  steps_per_execution=8)
    return model

def create_data_generators(train_path, test_path, batch_size, target_size, preprocessing_function):
    preprocess_func = preprocessing_function

    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_func,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        zoom_range=0.2,
        shear_range=0.2,
        fill_mode='nearest'
    )
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_func)

    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    steps_per_epoch = len(train_generator)
    validation_steps = len(test_generator)

    return DataGenerator(train_generator, steps_per_epoch), DataGenerator(test_generator, validation_steps), steps_per_epoch, validation_steps

def lr_schedule(epoch, lr):
    if epoch < 100:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

def main(find_lr=True, pretrain_model_eval=True, transfer_learning_model='Any'):

    # Mount Google Drive if using Colab
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        DATASET_PATH = '/content/drive/MyDrive/Colab Notebooks/datasets/1703138137_dataset/part_1/dataset_hist_structures'
    except ImportError:
        load_dotenv(verbose=True, dotenv_path='.env', override=True)
        DATASET_PATH = os.getenv('DATASET_PATH')

    try:
        ARCHITECTURES = {
            'ResNet50V2': ResNet50V2,
            'VGG16': VGG16,
            'InceptionV3': InceptionV3,
            'MobileNetV2': MobileNetV2,
            'EfficientNetB0': EfficientNetB0
        }

        ARCHITECTURE_INPUT_SHAPES = {
            'ResNet50V2': (128, 128, 3),
            'VGG16': (224, 224, 3),
            'InceptionV3': (299, 299, 3),
            'MobileNetV2': (224, 224, 3),
            'EfficientNetB0': (224, 224, 3)
        }

        PREPROCESSING_FUNCTIONS = {
            'ResNet50V2': resnet_preprocess,
            'VGG16': vgg_preprocess,
            'InceptionV3': inception_preprocess,
            'MobileNetV2': mobilenet_preprocess,
            'EfficientNetB0': efficientnet_preprocess
        }

        # ----------- HYPERPARAMETERS -----------
        #
        batch_size = 32
        # Learning Rate Epochs
        lr_epochs = 5
        # Default Learning Rate
        initial_learning_rate = 0.001
        # Early Stopping Patience
        patience = 5
        # Accuracy Callback
        target_accuracy = 0.95
        # Training Epochs
        trn_epochs = 20
        #
        # ----------- HYPERPARAMETERS -----------

        # Architecture - Change this to switch architectures
        tl_model = transfer_learning_model
        architecture = ARCHITECTURES[tl_model]
        input_shape = ARCHITECTURE_INPUT_SHAPES[architecture.__name__]
        target_size = ARCHITECTURE_INPUT_SHAPES[architecture.__name__][:2]
        preprocessing_function = PREPROCESSING_FUNCTIONS[architecture.__name__]

        # Set up dataset directories
        train_path = f'{DATASET_PATH}/structures_dataset'
        test_path = f'{DATASET_PATH}/dataset_test'

        train_counts = count_samples(train_path)
        test_counts = count_samples(test_path)

        print()
        print(f"TensorFlow version: {tf.__version__}")
        print(f"Eager execution: {tf.executing_eagerly()}")
        print()

        print()
        print("\nTraining samples per class:")
        for class_name, count in sorted(train_counts.items()):
            print(f"{class_name}: {count}")
        print()

        print()
        print("\nTest samples per class:")
        for class_name, count in sorted(test_counts.items()):
            print(f"{class_name}: {count}")
        print()

        # Create the model
        num_classes = len(train_counts)
        model = create_model(num_classes, architecture, input_shape)

        print()
        print(f"Using {architecture} architecture")
        print(model.summary())
        print()

        # Set up data generators
        train_generator, test_generator, steps_per_epoch, validation_steps = create_data_generators(
            train_path, test_path, batch_size=batch_size, target_size=target_size, preprocessing_function=preprocessing_function)

        print()
        print(f"Steps per epoch: {steps_per_epoch}")
        print(f"Validation steps: {validation_steps}")
        print()

        # Compile the model with a temporary learning rate
        compiled_model = compile_model(model, learning_rate=initial_learning_rate)

        if find_lr:
            # Find optimal learning rate
            print()
            print("Finding optimal learning rate...")
            print()
            lr_finder = LearningRateFinder(compiled_model)
            lr_finder.find(train_generator, start_lr=1e-7, end_lr=1, epochs=5)
            lr_finder.plot_loss()
            lr_finder.print_results()
            best_lr = lr_finder.get_best_lr()
            best_lr = float(best_lr)  # Convert to float
            print()
            print(f"Best initial learning rate: {best_lr}")
            print()
            # Recompile the model with the best learning rate
            print("Recompiling model with best learning rate...")
            print()
            compiled_model = compile_model(model, learning_rate=best_lr)
            
        else:
            best_lr = initial_learning_rate

        # Set up callbacks
        accuracy_callback = AccuracyCallback(target_accuracy=target_accuracy)
        debug_callback = DebugCallback()
        early_stopping = EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
        dataset_logger = DatasetLogger(train_generator, test_generator)
        custom_validation = CustomValidationCallback(test_generator, validation_steps)
        augmentation_callback = AugmentationCallback(train_generator)
        lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

        # Compute class weights
        class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.generator.classes), y=train_generator.generator.classes)
        class_weight_dict = dict(enumerate(class_weights))

        if pretrain_model_eval:
            # Separate evaluation before training
            print()
            print("Evaluating model before training:")
            print()
            evaluation = compiled_model.evaluate(test_generator.generator, steps=validation_steps)
            print()
            print(f"Initial evaluation: {evaluation}")
            print()
            
        try:
            # Training
            history = compiled_model.fit(
                train_generator.generator,
                steps_per_epoch=steps_per_epoch,
                epochs=trn_epochs,
                validation_data=test_generator.generator,
                validation_steps=validation_steps,
                callbacks=[accuracy_callback, augmentation_callback, custom_validation, debug_callback, dataset_logger, early_stopping, lr_callback],
                class_weight=class_weight_dict,
                verbose=1
            )

        except tf.errors.ResourceExhaustedError:
            print()
            print("Memory error occurred. Try reducing batch size or image dimensions.")
            print()
        except Exception as e:
            print()
            print(f"An error occurred during training: {e}")
            print()
            import traceback
            traceback.print_exc()

        # Final evaluation
        print()
        print("Final evaluation:")
        print("-----------------")
        final_evaluation = compiled_model.evaluate(test_generator.generator, steps=validation_steps)
        print()
        print(f"Final evaluation: {final_evaluation}")
        print()
    except Exception as e:
        print()
        print(f"An error occurred: {e}")
        print()
        import traceback
        traceback.print_exc()

In [ ]:
main(find_lr=True, pretrain_model_eval=True, transfer_learning_model='EfficientNetB0')

**Explanations:**
* This codebase implements a Convolutional Neural Network (CNN) using transfer learning with the ResNet50V2 architecture for image classification. It uses a dataset of architectural elements (e.g., altars, domes, columns) split into training and test sets. The model is compiled with Adam optimizer and categorical crossentropy loss. Data augmentation is applied to increase the diversity of training samples. The training process includes callbacks for early stopping, learning rate scheduling, and custom validation. The model's performance is evaluated and visualized over 20 epochs.

1. `DataGenerator` class:
  - Optimizable parameters:
    - None directly, but it relies on the ImageDataGenerator settings.

2. `CustomAugmentationLayer` class:
   - Optimizable parameters:
    - Contrast range (lower, upper)
    - Saturation range (lower, upper)
   - Role in optimization:
    - Adjusting these can increase data diversity and potentially reduce overfitting.

3. `create_model` function:
   - Optimizable parameters:
    - Number and size of dense layers
    - Dropout rate (currently 0.5)
    - L2 regularization strength
   - Role in optimization:
    - Adjusting model complexity can help balance underfitting and overfitting.

4. `compile_model` function:
   - Optimizable parameters:
    - Initial learning rate (currently 0.001)
    - Learning rate decay steps and rate
    - Choice of optimizer (currently Adam)
   - Role in optimization:
    - Proper learning rate and optimizer settings are crucial for efficient training and convergence.

5. `AccuracyCallback` class:
   - Optimizable parameters:
    - Target accuracy threshold
   - Role in optimization:
    - Adjusting this can prevent premature stopping or unnecessarily long training.

6. `count_samples` function:
   - Optimizable parameters:
    - None

7. `DebugCallback` class:
   - Optimizable parameters:
    - None, but frequency of debug prints could be adjusted.

8. `create_data_generators` function:
   - Optimizable parameters:
    - Batch size (currently 32)
    - Data augmentation parameters (rotation_range, width_shift_range, etc.)
   - Role in optimization:
    - Proper batch size and augmentation can improve training stability and reduce overfitting.

9. `DatasetLogger` class:
   - Optimizable parameters:
    - None

10. `CustomValidationCallback` class:
    - Optimizable parameters:
      - None, but custom metrics could be added.

11. `visualize_augmentation` function:
    - Optimizable parameters:
      - None

12. `AugmentationCallback` class:
    - Optimizable parameters:
     - Frequency of augmentation visualization
    - Role in optimization:
      - Adjusting this helps in monitoring augmentation effects without slowing training.

13. Main execution block:
    - Optimizable parameters:
      - Number of epochs (currently 20)
      - Early stopping patience (currently 10)
      - Class weight calculation method
    - Role in optimization:
      - These parameters affect overall training duration and handling of class imbalance.

Additional global optimizations:
1. Learning rate scheduling: Implement more sophisticated schedules like cyclic learning rates or warm restarts.
2. Model architecture: Experiment with different base models (e.g., EfficientNet, VGG) or custom architectures.
3. Regularization: Add or adjust L2 regularization, increase dropout rates, or implement other techniques like label smoothing.
4. Data preprocessing: Normalize input data, apply additional augmentation techniques like mixup or cutout.
5. Ensemble methods: Train multiple models and combine their predictions for improved reliability.
6. Cross-validation: Implement k-fold cross-validation for more robust performance estimation.
7. Gradient clipping: Add gradient clipping to prevent exploding gradients and stabilize training.
8. Batch normalization: Add batch normalization layers for improved training stability and potentially faster convergence.
9. Learning rate finder: Implement a learning rate finder to determine optimal initial learning rates.
10. Progressive resizing: Start training with smaller image sizes and gradually increase, potentially speeding up early training stages.

By carefully tuning these parameters and implementing these techniques, you can work towards optimizing the model's performance, improving its reliability, and finding the right balance between underfitting and overfitting for your specific architectural element classification task.

**Why It Is Important:**
* Using a CNN with transfer learning is important because it leverages pre-trained weights on a large dataset (ImageNet), allowing the model to learn high-level features more quickly and effectively, especially when dealing with a relatively small dataset. This approach often leads to better performance and faster convergence compared to training a model from scratch, making it particularly useful for specialized image classification tasks like identifying architectural elements.

**Observations:**
* The model quickly achieves high accuracy (>80%) within the first few epochs.
* Validation accuracy peaks around epoch 6 at about 86% and then fluctuates.
* There's some overfitting, as training accuracy consistently exceeds validation accuracy.
* The learning rate decreases steadily over the epochs, as designed by the exponential decay schedule.
* Both training and validation loss decrease rapidly initially and then plateau.
* The model's performance seems to stabilize after about 10-15 epochs.

**Conclusions:**
* The transfer learning approach is effective, allowing the model to achieve good performance quickly.
* The model reaches a reasonable accuracy (~83% on the validation set) for classifying architectural elements.
* There's room for improvement, as the model shows signs of overfitting and doesn't consistently improve after the initial rapid progress.
* The current learning rate schedule and data augmentation help, but may not be optimal for this specific task.

**Recommendations:**
* Experiment with stronger regularization techniques (e.g., increased dropout, L2 regularization) to combat overfitting.
* Try different learning rate schedules, such as cyclical learning rates or a more aggressive decay, to potentially improve convergence.
* Increase data augmentation to provide more diverse training samples and potentially reduce overfitting.
* Consider implementing techniques like gradient clipping to stabilize training.
* Explore ensemble methods or cross-validation to improve overall performance and reliability.
* Analyze misclassifications to identify patterns and potentially refine the model architecture or data preprocessing steps.

#### <a id='toc1_4_2_4_'></a>[**Visualize Training And Validation Accuracy**](#toc0_)

In [ ]:
# def plot_training_history(history, lr_logger):
def plot_training_history(history):

    # Plot training history and learning rate
    plt.figure(figsize=(15, 5))

    # Plot training and validation accuracy
    plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot training and validation loss
    plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # # Plot learning rate
    # plt.subplot(1, 3, 3)
    # plt.plot(range(1, len(lr_logger.lrs) + 1), lr_logger.lrs)
    # plt.title('Learning Rate over Epochs')
    # plt.xlabel('Epoch')
    # plt.ylabel('Learning Rate')
    # 
    # plt.tight_layout()
    # plt.show()

# plot_training_history(history, lr_logger)
plot_training_history(history)

**Explanations:**

- This code defines a function `plot_training_history` that visualizes the training and validation accuracy and loss over epochs. It then applies this function to the training history of the augmented model.

**Why It Is Important:**

- Visualizing the training and validation metrics helps us understand how the model is learning over time. It allows us to identify potential issues such as overfitting (when training accuracy continues to improve but validation accuracy plateaus or decreases) or underfitting (when both training and validation accuracy are low and not improving). This information is crucial for deciding whether to adjust the model architecture, change hyperparameters, or modify the training process.

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_2_5_'></a>[**Test Trained Model**](#toc0_)

In [ ]:
def predict_image_class(model, img_path, class_names, target_size, preprocessing_function):
    target_size = target_size  # Get the target size for the chosen architecture
    preprocess_func = preprocessing_function  # Get the preprocessing function for the chosen architecture

    # Load and preprocess the image
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_func(img_array)

    # Make prediction
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    predicted_class = class_names[predicted_class_index]
    confidence = predictions[0][predicted_class_index]

    return predicted_class, confidence

def visualize_prediction(img_path, target_size, predicted_class, confidence):
    img = image.load_img(img_path, target_size=target_size)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Predicted: {predicted_class}\nConfidence: {confidence:.2f}")
    plt.show()

# Save the class names
class_names = list(train_generator.generator.class_indices.keys())

# Test the model on new images
test_image_paths = [
    f'{DATASET_PATH}/model_test_images/gargoyle.jpg',
    f'{DATASET_PATH}/model_test_images/purdue-bell-tower.jpg',
    f'{DATASET_PATH}/model_test_images/purdue-memorial-union-stained-glass-2.jpg',
]
print()
print("\nTesting model on new images:")
print()

for img_path in test_image_paths:
    predicted_class, confidence = predict_image_class(compiled_model, img_path, class_names, target_size, preprocessing_function)
    print()
    print(f"Image: {img_path}")
    print(f"Predicted class: {predicted_class}")
    print(f"Confidence: {confidence:.2f}")
    print()

    # Visualize prediction
    visualize_prediction(img_path, target_size, predicted_class, confidence)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_2_3_'></a>[**Train The Model With Augmentation - v2**](#toc0_)

In [ ]:
import os
import warnings
from typing import Dict, Tuple, List
import logging
from dataclasses import dataclass

import yaml
import numpy as np
import tensorflow as tf
from dotenv import load_dotenv
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import backend as K
from tensorflow.keras.applications import (
    EfficientNetB0,
    InceptionV3,
    MobileNetV2,
    ResNet50V2,
    VGG16,
)
from tensorflow.keras.callbacks import (
    Callback,
    EarlyStopping,
    LambdaCallback,
    LearningRateScheduler,
)
from tensorflow.keras.layers import (
    BatchNormalization,
    Dense,
    Dropout,
    GlobalAveragePooling2D,
    Input,
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import Sequence

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Suppress warnings
warnings.filterwarnings("ignore")

# Load configuration
with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)

@dataclass
class ModelConfig:
    architecture: str
    input_shape: Tuple[int, int, int]

@dataclass
class TrainingConfig:
    batch_size: int
    initial_learning_rate: float
    epochs: int
    early_stopping_patience: int
    target_accuracy: float
    lr_epochs: int

model_config = ModelConfig(**config['model'])
training_config = TrainingConfig(**config['training'])

# Custom exceptions
class DatasetError(Exception):
    """Exception raised for errors in the dataset."""
    pass

class ModelError(Exception):
    """Exception raised for errors in the model creation or training."""
    pass

class AccuracyCallback(Callback):
    """Callback to stop training when a target accuracy is reached."""

    def __init__(self, target_accuracy: float):
        super().__init__()
        self.target_accuracy = target_accuracy

    def on_epoch_end(self, epoch: int, logs: Dict[str, float] = None) -> None:
        if logs.get('val_accuracy') >= self.target_accuracy:
            logger.info(f"\nReached {self.target_accuracy*100}% validation accuracy. Stopping training.")
            logger.info("\n--------------------\n")
            self.model.stop_training = True

class AugmentationCallback(Callback):
    """Callback to work with the CustomAugmentationLayer."""

    def __init__(self, train_generator: DataGenerator):
        super().__init__()
        self.train_generator = train_generator

    def on_batch_begin(self, batch: int, logs: Dict[str, float] = None) -> None:
        # You can add custom logic here if needed
        pass

class CustomValidationCallback(Callback):
    """Callback for custom validation after each epoch."""

    def __init__(self, validation_data: DataGenerator, validation_steps: int):
        super().__init__()
        self.validation_data = validation_data
        self.validation_steps = validation_steps

    def on_epoch_end(self, epoch: int, logs: Dict[str, float] = None) -> None:
        val_loss = 0
        val_accuracy = 0
        for i in range(self.validation_steps):
            x, y = next(self.validation_data.generator)
            val_metrics = self.model.test_on_batch(x, y)
            val_loss += val_metrics[0]
            val_accuracy += val_metrics[1]
        val_loss /= self.validation_steps
        val_accuracy /= self.validation_steps
        logs['val_loss'] = val_loss
        logs['val_accuracy'] = val_accuracy
        logger.info(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

class DatasetLogger(Callback):
    """Callback to log dataset information at the beginning of each epoch."""

    def __init__(self, train_generator: DataGenerator, val_generator: DataGenerator):
        super().__init__()
        self.train_generator = train_generator
        self.val_generator = val_generator

    def on_epoch_begin(self, epoch: int, logs: Dict[str, float] = None) -> None:
        logger.info(f"Epoch {epoch + 1} - Train samples: {len(self.train_generator.generator.classes)}")
        logger.info(f"Epoch {epoch + 1} - Val samples: {len(self.val_generator.generator.classes)}")
        logger.info("--------------------")

    def on_epoch_end(self, epoch: int, logs: Dict[str, float] = None) -> None:
        if logs is not None:
            logger.info(f"Epoch {epoch + 1} - Train accuracy: {logs.get('accuracy', 'N/A'):.4f}")
            logger.info(f"Epoch {epoch + 1} - Val accuracy: {logs.get('val_accuracy', 'N/A'):.4f}")
            logger.info("--------------------")

class DebugCallback(Callback):
    """Callback for debugging purposes, providing detailed information about the training process."""

    def on_epoch_begin(self, epoch: int, logs: Dict[str, float] = None) -> None:
        logger.info(f"\nStarting epoch {epoch + 1}")

    def on_batch_begin(self, batch: int, logs: Dict[str, float] = None) -> None:
        if batch % 50 == 0:
            logger.info(f"Starting batch {batch}")

    def on_epoch_end(self, epoch: int, logs: Dict[str, float] = None) -> None:
        logger.info(f"End of epoch {epoch + 1}")
        if logs:
            for key, value in logs.items():
                logger.info(f"{key}: {value}")
        logger.info("--------------------")

class LRLogger(Callback):
    def __init__(self):
        super().__init__()
        self.lrs = []

    def on_epoch_begin(self, epoch, logs=None):
        lr = float(K.get_value(self.model.optimizer.lr))
        self.lrs.append(lr)

class CustomAugmentationLayer(tf.keras.layers.Layer):
    """Custom augmentation layer for image preprocessing."""

    def call(self, inputs: tf.Tensor, training: bool = None) -> tf.Tensor:
        if training:
            augmented = tf.image.random_contrast(inputs, lower=0.8, upper=1.2)
            augmented = tf.image.random_saturation(augmented, lower=0.8, upper=1.2)
            return augmented
        return inputs

class DataGenerator(Sequence):
    """Custom data generator for feeding data to the model."""

    def __init__(self, generator, steps_per_epoch: int):
        self.generator = generator
        self.steps_per_epoch = steps_per_epoch

    def __len__(self) -> int:
        return self.steps_per_epoch

    def __getitem__(self, index: int) -> Tuple[np.ndarray, np.ndarray]:
        return next(self.generator)

    def on_epoch_end(self) -> None:
        self.generator.reset()

class LearningRateFinder:
    """Class to find the optimal learning rate."""

    def __init__(self, model: Model, stop_factor: float = 4, beta: float = 0.98):
        self.model = model
        self.stop_factor = stop_factor
        self.beta = beta
        self.lrs: List[float] = []
        self.losses: List[float] = []
        self.best_loss = float('inf')
        self.avg_loss = 0
        self.batch_num = 0
        self.weights_file = None

    def reset(self) -> None:
        """Reset the state of the learning rate finder."""
        self.lrs = []
        self.losses = []
        self.best_loss = float('inf')
        self.avg_loss = 0
        self.batch_num = 0
        self.weights_file = None

    def on_batch_end(self, batch: int, logs: Dict[str, float]) -> None:
        """Callback method to update learning rate and loss after each batch."""
        lr = self.model.optimizer.lr
        if hasattr(lr, 'numpy'):
            lr = lr.numpy()
        self.lrs.append(lr)

        loss = logs["loss"]
        self.batch_num += 1
        self.avg_loss = (self.beta * self.avg_loss) + ((1 - self.beta) * loss)
        smooth = self.avg_loss / (1 - (self.beta ** self.batch_num))
        self.losses.append(smooth)

        stop_loss = self.stop_factor * self.best_loss

        if self.batch_num > 1 and smooth < self.best_loss:
            self.best_loss = smooth

        if smooth > stop_loss:
            self.model.stop_training = True

        if self.batch_num % 100 == 0:
            logger.info(f"Batch {self.batch_num}: lr = {lr:.6f}, loss = {smooth:.4f}")

    def find(self, train_data: DataGenerator, start_lr: float, end_lr: float, batch_size: int = 32, epochs: int = 5) -> None:
        """Find the optimal learning rate."""
        self.reset()
        num_samples = len(train_data.generator.classes)
        steps_per_epoch = int(np.ceil(num_samples / batch_size))

        self.weights_file = "lrf_weights.h5"
        self.model.save_weights(self.weights_file)

        orig_lr = K.get_value(self.model.optimizer.lr)
        K.set_value(self.model.optimizer.lr, start_lr)

        logger.info(f"Starting Learning Rate Finder: LR range {start_lr} to {end_lr}")

        lr_schedule = lambda epoch, batch: start_lr * (end_lr / start_lr) ** (batch / (epochs * steps_per_epoch))

        callback = LambdaCallback(on_batch_end=lambda batch, logs: self.on_batch_end(batch, logs))
        self.model.fit(
            train_data.generator,
            steps_per_epoch=steps_per_epoch,
            epochs=epochs,
            callbacks=[callback, LearningRateScheduler(lr_schedule)],
            verbose=0
        )

        logger.info("Learning Rate Finder complete")

        self.model.load_weights(self.weights_file)
        K.set_value(self.model.optimizer.lr, orig_lr)

    def plot_loss(self) -> None:
        """Plot the loss against the learning rate."""
        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 6))
        plt.plot(self.lrs, self.losses)
        plt.xscale("log")
        plt.xlabel("Learning rate")
        plt.ylabel("Loss")
        plt.title("Learning Rate vs. Loss")
        plt.show()

    def get_best_lr(self) -> float:
        """Get the best learning rate based on the minimum gradient of the loss curve."""
        min_grad_idx = np.argmin(np.gradient(np.array(self.losses)))
        return self.lrs[min_grad_idx]

    def print_results(self) -> None:
        """Print the results of the learning rate finder."""
        logger.info("\nLearning Rate Finder Results:")
        logger.info(f"Minimum loss: {min(self.losses):.4f}")
        logger.info(f"Maximum loss: {max(self.losses):.4f}")
        logger.info(f"Learning rate range: {min(self.lrs):.6f} to {max(self.lrs):.6f}")

def count_samples(dataset_path: str) -> Dict[str, int]:
    """
    Count the number of samples in each class of the dataset.

    Args:
        dataset_path (str): Path to the dataset directory.

    Returns:
        Dict[str, int]: Dictionary with class names as keys and sample counts as values.
    """
    class_counts = {}
    try:
        for class_name in os.listdir(dataset_path):
            class_path = os.path.join(dataset_path, class_name)
            if os.path.isdir(class_path):
                class_counts[class_name] = len(os.listdir(class_path))
    except OSError as e:
        raise DatasetError(f"Error reading dataset directory: {e}")
    return class_counts

def create_model(num_classes: int, architecture: callable, input_shape: Tuple[int, int, int]) -> Model:
    """
    Create a transfer learning model based on the given architecture.

    Args:
        num_classes (int): Number of classes in the dataset.
        architecture (callable): Base model architecture (e.g., ResNet50V2).
        input_shape (Tuple[int, int, int]): Input shape for the model.

    Returns:
        Model: Compiled Keras model.
    """
    base_model = architecture(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    for layer in base_model.layers:
        layer.trainable = False

    inputs = Input(shape=input_shape)
    x = CustomAugmentationLayer()(inputs)
    x = base_model(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = Dropout(0.5)(x)
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    output = Dense(num_classes, activation='softmax')(x)

    return Model(inputs=inputs, outputs=output)

def compile_model(model: Model, learning_rate: float) -> Model:
    """
    Compile the model with the given learning rate.

    Args:
        model (Model): The model to compile.
        learning_rate (float): The initial learning rate.

    Returns:
        Model: Compiled Keras model.
    """
    lr_schedule = ExponentialDecay(
        learning_rate,
        decay_steps=100,
        decay_rate=0.96,
        staircase=True)

    optimizer = Adam(learning_rate=lr_schedule)

    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        steps_per_execution=8
    )
    return model

def create_data_generators(train_path: str, test_path: str, batch_size: int, target_size: Tuple[int, int], preprocessing_function: callable) -> Tuple[DataGenerator, DataGenerator, int, int]:
    """
    Create data generators for training and testing.

    Args:
        train_path (str): Path to the training data.
        test_path (str): Path to the test data.
        batch_size (int): Batch size for training.
        target_size (Tuple[int, int]): Target size for the input images.
        preprocessing_function (callable): Preprocessing function for the images.

    Returns:
        Tuple[DataGenerator, DataGenerator, int, int]: Train generator, test generator, steps per epoch, and validation steps.
    """
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocessing_function,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        zoom_range=0.2,
        shear_range=0.2,
        fill_mode='nearest'
    )
    test_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )

    test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )

    steps_per_epoch = len(train_generator)
    validation_steps = len(test_generator)

    return DataGenerator(train_generator, steps_per_epoch), DataGenerator(test_generator, validation_steps), steps_per_epoch, validation_steps

def lr_schedule(epoch: int, lr: float) -> float:
    """
    Learning rate schedule function.

    Args:
        epoch (int): Current epoch number.
        lr (float): Current learning rate.

    Returns:
        float: Updated learning rate.
    """
    if epoch < 100:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

from typing import Dict, Any

def validate_config(config: Dict[str, Any], valid_architectures: Dict[str, callable]) -> None:
    """
    Validate the entire configuration.

    Args:
        config (Dict[str, Any]): The entire configuration dictionary.
        valid_architectures (Dict[str, callable]): Dictionary of valid architecture names and their corresponding functions.

    Raises:
        ValueError: If any configuration parameter is invalid.
    """
    # Validate model configuration
    validate_model_config(config.get('model', {}), valid_architectures)

    # Validate training configuration
    validate_training_config(config.get('training', {}))

    # Validate data configuration
    validate_data_config(config.get('data', {}))

def validate_model_config(model_config: Dict[str, Any], valid_architectures: Dict[str, callable]) -> None:
    """Validate the model configuration."""
    if 'architecture' not in model_config:
        raise ValueError("Model configuration must include 'architecture'")
    
    architecture = model_config['architecture']
    if architecture not in valid_architectures:
        raise ValueError(f"Invalid architecture: {architecture}. Valid options are: {', '.join(valid_architectures.keys())}")

def validate_training_config(training_config: Dict[str, Any]) -> None:
    """Validate the training configuration."""
    required_fields = ['batch_size', 'initial_learning_rate', 'epochs', 'early_stopping_patience', 'target_accuracy']
    for field in required_fields:
        if field not in training_config:
            raise ValueError(f"Training configuration must include '{field}'")

    if training_config['batch_size'] <= 0:
        raise ValueError(f"Invalid batch size: {training_config['batch_size']}. Must be a positive integer.")
    
    if training_config['initial_learning_rate'] <= 0:
        raise ValueError(f"Invalid initial learning rate: {training_config['initial_learning_rate']}. Must be a positive float.")
    
    if training_config['epochs'] <= 0:
        raise ValueError(f"Invalid number of epochs: {training_config['epochs']}. Must be a positive integer.")
    
    if training_config['early_stopping_patience'] < 0:
        raise ValueError(f"Invalid early stopping patience: {training_config['early_stopping_patience']}. Must be a non-negative integer.")
    
    if not 0 < training_config['target_accuracy'] <= 1:
        raise ValueError(f"Invalid target accuracy: {training_config['target_accuracy']}. Must be between 0 and 1.")

def validate_data_config(data_config: Dict[str, Any]) -> None:
    """Validate the data configuration."""
    required_fields = ['train_path', 'test_path']
    for field in required_fields:
        if field not in data_config:
            raise ValueError(f"Data configuration must include '{field}'")

    if not os.path.exists(data_config['train_path']):
        raise ValueError(f"Train path does not exist: {data_config['train_path']}")
    
    if not os.path.exists(data_config['test_path']):
        raise ValueError(f"Test path does not exist: {data_config['test_path']}")

def main(find_lr: bool = True, pretrain_model_eval: bool = True):
    """
    Main function to run the training pipeline.

    Args:
        find_lr (bool): Whether to find the optimal learning rate.
        pretrain_model_eval (bool): Whether to evaluate the model before training.
    """
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        DATASET_PATH = '/content/drive/MyDrive/Colab Notebooks/datasets/1703138137_dataset/part_1/dataset_hist_structures'
    except ImportError:
        load_dotenv(verbose=True, dotenv_path='.env', override=True)
        DATASET_PATH = os.getenv('DATASET_PATH')

    ARCHITECTURES = {
        'ResNet50V2': ResNet50V2,
        'VGG16': VGG16,
        'InceptionV3': InceptionV3,
        'MobileNetV2': MobileNetV2,
        'EfficientNetB0': EfficientNetB0
    }

    ARCHITECTURE_INPUT_SHAPES = {
        'ResNet50V2': (128, 128, 3),
        'VGG16': (224, 224, 3),
        'InceptionV3': (299, 299, 3),
        'MobileNetV2': (224, 224, 3),
        'EfficientNetB0': (224, 224, 3)
    }

    PREPROCESSING_FUNCTIONS = {
        'ResNet50V2': tf.keras.applications.resnet_v2.preprocess_input,
        'VGG16': tf.keras.applications.vgg16.preprocess_input,
        'InceptionV3': tf.keras.applications.inception_v3.preprocess_input,
        'MobileNetV2': tf.keras.applications.mobilenet_v2.preprocess_input,
        'EfficientNetB0': tf.keras.applications.efficientnet.preprocess_input
    }

    # Load configuration
    with open('config.yaml', 'r') as config_file:
        config = yaml.safe_load(config_file)

    # Validate the entire configuration
    try:
        validate_config(config, ARCHITECTURES)
    except ValueError as e:
        logger.error(f"Configuration error: {e}")
        return

    # Extract configuration values
    model_config = config['model']
    training_config = config['training']
    data_config = config['data']

    architecture = ARCHITECTURES[model_config['architecture']]
    input_shape = ARCHITECTURE_INPUT_SHAPES[architecture_name]
    target_size = input_shape[:2]
    preprocessing_function = PREPROCESSING_FUNCTIONS[architecture_name]

    train_path = os.path.join(DATASET_PATH, data_config['train_path'])
    test_path = os.path.join(DATASET_PATH, data_config['test_path']))
        
    try:
        train_counts = count_samples(train_path)
        test_counts = count_samples(test_path)
    except DatasetError as e:
        logger.error(f"Error in dataset: {e}")
        return

    logger.info(f"TensorFlow version: {tf.__version__}")
    logger.info(f"Eager execution: {tf.executing_eagerly()}")

    logger.info("\nTraining samples per class:")
    for class_name, count in sorted(train_counts.items()):
        logger.info(f"{class_name}: {count}")

    logger.info("\nTest samples per class:")
    for class_name, count in sorted(test_counts.items()):
        logger.info(f"{class_name}: {count}")

    num_classes = len(train_counts)
    model = create_model(num_classes, architecture, input_shape)

    logger.info(f"Using {architecture.__name__} architecture")
    model.summary()

    train_generator, test_generator, steps_per_epoch, validation_steps = create_data_generators(
        train_path, test_path, batch_size=training_config.batch_size, target_size=target_size, preprocessing_function=preprocessing_function)

    logger.info(f"Steps per epoch: {steps_per_epoch}")
    logger.info(f"Validation steps: {validation_steps}")

    compiled_model = compile_model(model, learning_rate=training_config.initial_learning_rate)

    if find_lr:
        logger.info("Finding optimal learning rate...")
        lr_finder = LearningRateFinder(compiled_model)
        lr_finder.find(train_generator, start_lr=1e-7, end_lr=1, epochs=training_config.lr_epochs)
        lr_finder.plot_loss()
        lr_finder.print_results()
        best_lr = lr_finder.get_best_lr()
        logger.info(f"Best initial learning rate: {best_lr}")
        logger.info("Recompiling model with best learning rate...")
        compiled_model = compile_model(model, learning_rate=best_lr)
    else:
        best_lr = training_config.initial_learning_rate

    accuracy_callback = AccuracyCallback(target_accuracy=training_config.target_accuracy)
    debug_callback = DebugCallback()
    early_stopping = EarlyStopping(monitor='val_loss', patience=training_config.early_stopping_patience, restore_best_weights=True)
    dataset_logger = DatasetLogger(train_generator, test_generator)
    custom_validation = CustomValidationCallback(test_generator, validation_steps)
    augmentation_callback = AugmentationCallback(train_generator)
    lr_callback = LearningRateScheduler(lr_schedule)
    lr_logger = LRLogger()

    class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.generator.classes), y=train_generator.generator.classes)
    class_weight_dict = dict(enumerate(class_weights))

    if pretrain_model_eval:
        logger.info("Evaluating model before training:")
        evaluation = compiled_model.evaluate(test_generator.generator, steps=validation_steps)
        logger.info(f"Initial evaluation: {evaluation}")

    try:
        history = compiled_model.fit(
            train_generator.generator,
            steps_per_epoch=steps_per_epoch,
            epochs=training_config.epochs,
            validation_data=test_generator.generator,
            validation_steps=validation_steps,
            callbacks=[
                accuracy_callback,
                debug_callback,
                early_stopping,
                dataset_logger,
                custom_validation,
                augmentation_callback,
                lr_callback,
                lr_logger
            ],
            class_weight=class_weight_dict,
            verbose=1
        )

    except tf.errors.ResourceExhaustedError:
        logger.error("Memory error occurred. Try reducing batch size or image dimensions.")
    except Exception as e:
        logger.error(f"An error occurred during training: {e}")
        import traceback
        logger.error(traceback.format_exc())

    logger.info("Final evaluation:")
    final_evaluation = compiled_model.evaluate(test_generator.generator, steps=validation_steps)
    logger.info(f"Final evaluation: {final_evaluation}")

if __name__ == "__main__":
    main()

#### <a id='toc1_4_2_4_'></a>[**Visualize Training And Validation Accuracy**](#toc0_)

#### <a id='toc1_4_2_4_'></a>[**Test Trained Model**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_2_3_'></a>[**Next Section**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

### <a id='toc1_4_3_'></a>[**4.3. Forecasting using machine learning algorithms**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_3_1_'></a>[**4.3.1. Forecasting using machine learning algorithms**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

##### <a id='toc1_4_3_1_1_'></a>[**4.3.1.1. Generate necessary features for the development of these models, like day of the week, quarter of the year, month, year, day of the month and so on**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

##### <a id='toc1_4_3_1_2_'></a>[**4.3.1.2. Use the data from the last six months as the testing data**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

##### <a id='toc1_4_3_1_3_'></a>[**4.3.1.3. Compute the root mean square error (RMSE) values for each model to compare their performances**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

##### <a id='toc1_4_3_1_4_'></a>[**4.3.1.4. Use the best-performing models to make a forecast for the next year**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

### <a id='toc1_4_4_'></a>[**4.4. Forecasting using deep learning algorithms**](#toc0_)

#### <a id='toc1_4_4_1_'></a>[**4.4.1. Use sales amount for predictions instead of item count**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_4_2_'></a>[**4.4.2. Build a long short-term memory (LSTM) model for predictions**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

##### <a id='toc1_4_4_2_1_'></a>[**4.4.2.1. Define the train and test series**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

##### <a id='toc1_4_4_2_2_'></a>[**4.4.2.2. Generate synthetic data for the last 12 months**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

##### <a id='toc1_4_4_2_3_'></a>[**4.4.2.3. Build and train an LSTM model**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

##### <a id='toc1_4_4_2_4_'></a>[**4.4.2.4. Use the model to make predictions for the test data**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_4_3_'></a>[**4.4.3. Calculate the mean absolute percentage error (MAPE) and comment on the model's performance**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]

#### <a id='toc1_4_4_4_'></a>[**4.4.4. Develop another model using the entire series for training, and use it to forecast for the next three months**](#toc0_)

**Explanations:**

- [Placeholder for observations after running the code]

**Why It Is Important:**

- [Placeholder for observations after running the code]

**Observations:**

- [Placeholder for observations after running the code]

**Conclusions:**

- [Placeholder for conclusions based on initial data view]

**Recommendations:**

- [Placeholder for recommendations based on initial data examination]